## Pricing and Market Data

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sovai as sov

sov.token_auth(token="your_token_here")

#### Processed Dataset

In [2]:
df_price = sov.data("market/prices", tickers=["MSFT","TSLA", "AMZN", "DDD", "JPM","GS","F","A"])

In [ ]:
import functions_framework
import pandas as pd
import gcsfs
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage
from google.oauth2 import service_account

@functions_framework.cloud_event
def hello_gcs(cloud_event):
    data = cloud_event.data

    event_id = cloud_event["id"]
    event_type = cloud_event["type"]

    bucket = data["bucket"]
    name = data["name"]
    metageneration = data["metageneration"]
    timeCreated = data["timeCreated"]
    updated = data["updated"]
    
    # Read the chunk from GCS
    chunk_path = f"gs://{bucket}/{name}"
    df_chunk = pd.read_parquet(chunk_path, engine='pyarrow')

    df_chunk = df_chunk.sort_values(["ticker","date"])
    
    
    # # Write the partitioned files to GCS
    # scopes = ['https://www.googleapis.com/auth/devstorage.read_write']
    # credentials = service_account.Credentials.from_service_account_info(
    #     service_account_info, scopes=scopes
    # )

    # gcs_filesystem = gcsfs.GCSFileSystem(token=credentials)

    gcs_filesystem = gcsfs.GCSFileSystem()


    row_group_size = 50000000

    table = pa.Table.from_pandas(df_chunk)

    name = "prices.parq"

    pq.write_to_dataset(
        table,
        root_path=f'gs://sovai-partitioned/sovai-accounting/{name}',
        partition_cols=["ticker"],
        filesystem=gcs_filesystem,
        row_group_size=row_group_size,
        use_threads=True,
        existing_data_behavior='overwrite_or_ignore',
        max_partitions=250000,
        basename_template='prices_{i}.parquet',
    )

    df_chunk_close = df_chunk[["closeadj"]].copy()

    df_chunk_close.to_parquet("gs://sovai-accounting/dataframes/prices_closeadj.parq")
    

    table = pa.Table.from_pandas(df_chunk)

    name = "prices_closeadj.parq"

    pq.write_to_dataset(
        table,
        root_path=f'gs://sovai-partitioned/sovai-accounting/{name}',
        partition_cols=["ticker"],
        filesystem=gcs_filesystem,
        row_group_size=row_group_size,
        use_threads=True,
        existing_data_behavior='overwrite_or_ignore',
        max_partitions=250000,
        basename_template='prices_closeadj_{i}.parquet',
    )


In [ ]:
  for d, n, p in zip(other_frames, names, parq):
      query = "SHARADAR/{}.csv?qopts.export=true&api_key={}".format(d, api)
      df = extract_pandas(query)
      if d == "TICKERS":
          df["lastupdated"] = pd.to_datetime(df["lastupdated"], infer_datetime_format=True)
      else:
          df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)
          df["lastupdated"] = pd.to_datetime(df["lastupdated"], infer_datetime_format=True)
          df = df.sort_values(["ticker", "date"])
      create_table = n
      df.to_parquet(f"gs://sovai-accounting/dataframes/{p}.parq")
      pandas_to_bq(df=df, client=client_bg, table_name=create_table)
      print("success {}".format(create_table))

In [3]:
df_price

open    high     low   close       volume  closeadj  \
ticker date                                                                
A      1999-11-18  45.500  50.000  40.000  44.000 44739900.000    26.968   
       1999-11-19  42.940  43.000  39.810  40.380 10897100.000    24.749   
       1999-11-22  41.310  44.000  40.060  44.000  4705200.000    26.968   
       1999-11-23  42.500  43.630  40.250  40.250  4274400.000    24.669   
       1999-11-24  40.130  41.940  40.000  41.060  3464400.000    25.166   
...                   ...     ...     ...     ...          ...       ...   
TSLA   2024-05-07 182.400 183.260 177.400 177.810 74397104.000   177.810   
       2024-05-08 171.590 176.060 170.150 174.720 79623280.000   174.720   
       2024-05-09 175.010 175.620 171.370 171.970 65583064.000   171.970   
       2024-05-10 173.050 173.060 167.750 168.470 72027744.000   168.470   
       2024-05-13 170.000 175.400 169.000 171.890 66499976.000   171.890   

                   closeunadj  
ticker date                    
A      1999-11-18      44.000  
       1999-11-19      40.380  
       1999-11-22      44.000  
       1999-11-23      40.250  
       1999-11-24      41.060  
...                       ...  
TSLA   2024-05-07     177.810  
       2024-05-08     174.720  
       2024-05-09     171.970  
       2024-05-10     168.470  
       2024-05-13     171.890  

[46315 rows x 7 columns]

In [4]:
!pip install polars_talib

ERROR: Could not find a version that satisfies the requirement polars_talib (from versions: none)
ERROR: No matching distribution found for polars_talib


In [36]:
import polars as pl
from polars_ta.prefix.tdx import *
from polars_ta.prefix.wq import *


df_polars = pl.from_pandas(df_price)


In [38]:
df_polars.shape

(48751, 9)

In [32]:
# def func_ts_date(df_polars: pl.DataFrame) -> pl.DataFrame:
#     # 时序指标计算前一定要保证有序
#     df_polars = df_polars.sort(by=['date'])
#     # 演示生成大量指标
#     df_polars = df_polars.with_columns([
#         *[ts_returns(df_polars['close'], i).alias(f'ROCP_{i:03d}') for i in (1, 3, 5, 10, 20, 60, 120)],
#         *[ts_mean(df_polars['close'], i).alias(f'SMA_{i:03d}') for i in (5, 10, 20, 60, 120)],
#         *[ts_std_dev(df_polars['close'], i).alias(f'STD_{i:03d}') for i in (5, 10, 20, 60, 120)],
#         *[ts_max(df_polars['high'], i).alias(f'HHV_{i:03d}') for i in (5, 10, 20, 60, 120)],
#         *[ts_min(df_polars['low'], i).alias(f'LLV_{i:03d}') for i in (5, 10, 20, 60, 120)],
#         *[ts_RSI(df_polars['close'], i).alias(f'RSI_{i:03d}') for i in (6, 12, 24)],
#     ])

#     return df

# # 多资产需要先按资产分组
# df_polars = df_polars.group_by(by=['ticker']).map_groups(func_ts_date)


In [39]:
import polars as pl
from polars_ta.prefix.tdx import *  # Assuming these functions work with Polars
from polars_ta.prefix.wq import *

def func_ts_date(df_group: pl.DataFrame) -> pl.DataFrame:
    # Ensure the data is sorted by date
    df_group = df_group.sort(by=['date'])
    
    # Calculate technical indicators
    df_group = df_group.with_columns([
        *[ts_returns(df_group['close'], i).alias(f'ROCP_{i:03d}') for i in (1, 3, 5, 10, 20, 60, 120)],
        *[ts_mean(df_group['close'], i).alias(f'SMA_{i:03d}') for i in (5, 10, 20, 60, 120)],
        *[ts_std_dev(df_group['close'], i).alias(f'STD_{i:03d}') for i in (5, 10, 20, 60, 120)],
        *[ts_max(df_group['high'], i).alias(f'HHV_{i:03d}') for i in (5, 10, 20, 60, 120)],
        *[ts_min(df_group['low'], i).alias(f'LLV_{i:03d}') for i in (5, 10, 20, 60, 120)],
        *[ts_RSI(df_group['close'], i).alias(f'RSI_{i:03d}') for i in (6, 12, 24)],
    ])

    return df_group

# Assuming df_polars is your initial DataFrame
df_polars_group = df_polars.group_by("ticker").apply(func_ts_date)


In [42]:
df_pandas = df_polars_group.to_pandas()

In [43]:
df_pandas

,ticker,date,open,high,low,close,volume,closeadj,closeunadj,ROCP_001,ROCP_003,ROCP_005,ROCP_010,ROCP_020,ROCP_060,ROCP_120,SMA_005,SMA_010,SMA_020,SMA_060,SMA_120,STD_005,STD_010,STD_020,STD_060,STD_120,HHV_005,HHV_010,HHV_020,HHV_060,HHV_120,LLV_005,LLV_010,LLV_020,LLV_060,LLV_120,RSI_006,RSI_012,RSI_024
0,A,1999-11-18,45.500000,50.000000,40.000000,44.000000,44739900.0,27.011000,44.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,1999-11-19,42.939999,43.000000,39.810001,40.380001,10897100.0,24.789000,40.380001,-0.082273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,1999-11-22,41.310001,44.000000,40.060001,44.000000,4705200.0,27.011000,44.000000,0.089648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,1999-11-23,42.500000,43.630001,40.250000,40.250000,4274400.0,24.709000,40.250000,-0.085227,-0.085227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,1999-11-24,40.130001,41.939999,40.000000,41.060001,3464400.0,25.205999,41.060001,0.020124,0.016840,NaN,NaN,NaN,NaN,NaN,41.938000,NaN,NaN,NaN,NaN,1.705918,NaN,NaN,NaN,NaN,50.00,NaN,NaN,NaN,NaN,39.810001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48746,F,2024-03-01,12.530000,12.640000,12.320000,12.450000,48409940.0,12.450000,12.450000,0.000804,0.037500,0.025535,-0.005591,0.028926,0.163551,0.012195,12.228009,12.209030,12.346931,11.867467,11.578649,0.213326,0.158384,0.340019,0.561935,0.864972,12.64,12.64,13.07,13.07,13.07,11.950000,11.95,11.54,10.540,9.63,0.635552,0.576720,0.560189
48747,F,2024-03-04,12.570000,13.030000,12.570000,12.740000,81561360.0,12.740000,12.740000,0.023293,0.035772,0.066109,0.035772,0.049423,0.201887,0.042553,12.386009,12.253030,12.376931,11.903134,11.582982,0.239807,0.224750,0.346808,0.548079,0.869463,13.03,13.03,13.07,13.07,13.07,11.955000,11.95,11.54,10.672,9.63,0.749285,0.637584,0.590131
48748,F,2024-03-05,12.650000,12.738000,12.500000,12.580000,47184820.0,12.580000,12.580000,-0.012559,0.011254,0.048333,0.026939,0.085418,0.176801,0.010442,12.502009,12.286030,12.426432,11.934634,11.584065,0.147530,0.245168,0.298188,0.531510,0.870622,13.03,13.03,13.07,13.07,13.07,11.955000,11.95,11.62,10.680,9.63,0.620983,0.586799,0.567874
48749,F,2024-03-06,12.520000,12.570000,12.330000,12.380000,48612440.0,12.380000,12.380000,-0.015898,-0.005622,0.006504,0.019769,0.025684,0.144177,-0.020570,12.518010,12.310030,12.441931,11.960634,11.581899,0.127753,0.241430,0.287135,0.514234,0.868315,13.03,13.03,13.07,13.07,13.07,12.320000,11.95,11.95,10.790,9.63,0.494079,0.529307,0.541247


In [14]:
from tsfeatures import tsfeatures

df_ml_features = tsfeatures(df_price.rename(columns={"ticker":"unique_id","date":"ds","close":"y"}), freq=7, dict_freqs={'D': 7, 'W': 4})

/breakout/median
Requesting URL: https://fastapi-sovai-y7o724zyfq-uc.a.run.app/breakout/median with params: None
Response Status: 200
Response Content-Type: application/json
tickers is None
None
tickers is: 
All ticker Initialized
https://s3.wasabisys.com/sovai-breakout-price/frames/df_test_summary_median.park?AWSAccessKeyId=KI07G4DK9XP5EC0NH1VU&Signature=kMPRpgrG%2F0iWAgSnyw7RfO%2FPcaM%3D&Expires=1709940569  (download link)
File is up-to-date, no need to download.
/breakout/median
Requesting URL: https://fastapi-sovai-y7o724zyfq-uc.a.run.app/breakout/median with params: None
Response Status: 200
Response Content-Type: application/json
tickers is None
None
tickers is: 
All ticker Initialized
https://s3.wasabisys.com/sovai-breakout-price/frames/df_test_summary_median.park?AWSAccessKeyId=KI07G4DK9XP5EC0NH1VU&Signature=kMPRpgrG%2F0iWAgSnyw7RfO%2FPcaM%3D&Expires=1709940569  (download link)
File is up-to-date, no need to download.
/breakout/median
Requesting URL: https://fastapi-sovai-y7o72

In [15]:
df_ml_features

,unique_id,hurst,series_length,unitroot_pp,unitroot_kpss,hw_alpha,hw_beta,hw_gamma,stability,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,peak,trough,x_pacf5,diff1x_pacf5,diff2x_pacf5,seas_pacf,nonlinearity,lumpiness,alpha,beta,arch_acf,garch_acf,arch_r2,garch_r2,flat_spots,entropy,crossing_points,arch_lm,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1
0,A,0.964467,6113,-1.914097,30.923481,0.967490,1.246224e-14,3.927633e-15,0.996996,1,7,0.998933,1.760746e-12,57.360481,42.790799,0.321530,0.346283,0.138194,7,3,0.999783,0.008232,0.484610,0.035851,0.018530,0.000254,0.967640,7.111212e-19,NaN,NaN,NaN,NaN,540,0.195805,74,0.992967,0.998429,9.849774,-0.032719,0.012393,-0.517720,0.273181,0.990387
1,AMZN,0.956214,6588,1.516708,39.116778,0.977355,1.937332e-07,3.618954e-19,0.999083,1,7,0.999668,4.939376e-14,65.929654,38.374481,0.289458,0.253810,0.154310,4,2,0.998805,0.003248,0.507443,0.015337,0.034228,0.000011,0.979121,0.000000e+00,NaN,NaN,NaN,NaN,4002,0.197457,19,0.996317,0.999058,9.899971,-0.020730,0.006462,-0.508479,0.273010,0.993601
2,DDD,0.916999,6588,-10.710873,12.357206,0.989505,4.125779e-13,4.455906e-14,0.997763,1,7,0.998695,8.904368e-13,30.901818,-22.790799,0.286906,0.272011,0.150281,4,4,1.000656,0.004685,0.508232,0.023621,0.007933,0.000255,0.989420,2.966716e-14,NaN,NaN,NaN,NaN,3264,0.283921,74,0.992743,0.998569,9.830776,-0.011245,0.016399,-0.520635,0.295682,0.989102
3,F,1.003472,6588,-8.661925,17.682880,0.913786,2.380496e-16,2.598120e-15,0.998846,1,7,0.999089,1.806993e-12,-42.514469,47.294285,0.233839,0.209441,0.173100,4,5,1.001426,0.009031,0.563333,0.014527,0.067681,0.000553,0.913711,2.593864e-10,NaN,NaN,NaN,NaN,754,0.271548,128,0.992955,0.998545,9.860471,-0.081858,0.014477,-0.528980,0.293309,0.991267
4,GS,1.001166,6252,-3.022506,35.801877,0.985401,6.153205e-15,0.000000e+00,0.998012,1,7,0.999143,1.058563e-13,66.082302,19.208319,0.303349,0.262756,0.202131,1,6,0.997218,0.001130,0.493071,0.000209,0.010636,0.000021,0.985101,0.000000e+00,NaN,NaN,NaN,NaN,302,0.294732,105,0.994889,0.998438,9.831354,-0.015020,0.005308,-0.510471,0.272680,0.989277
5,JPM,0.956608,6588,0.900368,39.222986,0.943022,2.623349e-19,2.137017e-03,0.998211,1,7,0.999399,7.769907e-14,65.349464,41.080177,0.281687,0.241012,0.174533,7,6,0.998052,0.006123,0.528983,0.021400,0.002410,0.000014,0.941796,0.000000e+00,NaN,NaN,NaN,NaN,870,0.207427,99,0.996290,0.998555,9.848616,-0.061679,0.015607,-0.544683,0.334570,0.990376
6,MSFT,0.917527,6588,5.238941,32.320475,0.894142,1.018872e-03,0.000000e+00,0.998478,1,7,0.999760,2.258070e-14,59.522554,48.146930,0.247192,0.264941,0.192070,7,6,0.999241,0.013097,0.558623,0.026709,0.004853,0.000003,0.893760,1.016708e-03,NaN,NaN,NaN,NaN,3948,0.249389,51,0.998002,0.998674,9.861089,-0.096976,0.035858,-0.531635,0.347619,0.991110
7,TSLA,1.036623,3446,-3.596809,22.823343,0.973896,3.538437e-14,2.119898e-15,0.999238,1,7,0.999001,1.335634e-12,45.630180,23.278764,0.288074,0.226735,0.170841,2,7,1.002935,0.004972,0.521210,-0.001841,0.089110,0.000067,0.973395,8.432647e-15,NaN,NaN,NaN,NaN,2413,0.206237,49,0.986153,0.998716,9.854182,-0.027212,0.023382,-0.518789,0.326838,0.990749


In [52]:
nasdaqdatalink.get_table('SHARADAR/SEP', ticker=['AAPL','TSLA'])data

/market/prices
Requesting URL: https://fastapi-sovai-y7o724zyfq-uc.a.run.app/market/prices with params: {'tickers': 'MSFT,TSLA,AMZN,DDD,JPM,GS,F', 'from_date': '2008-03-30T00:00:00'}
Cache entry purged.
Response Status: 200
Response Content-Type: application/octet-stream
tickers is: ['MSFT', 'TSLA', 'AMZN', 'DDD', 'JPM', 'GS', 'F']
{'tickers': 'MSFT,TSLA,AMZN,DDD,JPM,GS,F', 'from_date': '2008-03-30T00:00:00'}
tickers is: ['MSFT', 'TSLA', 'AMZN', 'DDD', 'JPM', 'GS', 'F']
The function took 13.864719867706299 seconds to execute.


In [50]:
df_price

,ticker,date,open,high,low,close,volume,closeadj,closeunadj
2300965,AMZN,2008-03-31,3.513000,3.582000,3.481000,3.565000,109406000.0,3.565000,71.300003
2300966,AMZN,2008-04-01,3.746000,3.855000,3.638000,3.835000,194680000.0,3.835000,76.699997
2300967,AMZN,2008-04-02,3.854000,3.950000,3.806000,3.869000,171848000.0,3.869000,77.370003
2300968,AMZN,2008-04-03,3.758000,3.801000,3.691000,3.747000,162998000.0,3.747000,74.940002
2300969,AMZN,2008-04-04,3.763000,3.891000,3.714000,3.844000,143056000.0,3.844000,76.870003
...,...,...,...,...,...,...,...,...,...
39000728,TSLA,2024-03-01,200.520004,204.520004,198.500000,202.639999,81867976.0,202.639999,202.639999
39000729,TSLA,2024-03-04,198.729996,199.750000,186.720001,188.139999,133227896.0,188.139999,188.139999
39000730,TSLA,2024-03-05,183.050003,184.589996,177.570007,180.740005,119184792.0,180.740005,180.740005
39000731,TSLA,2024-03-06,179.990005,181.576004,173.699997,176.539993,106624576.0,176.539993,176.539993


In [15]:
service_account_info = {}


filters = []

In [11]:


tickers=["MSFT","TSLA", "AMZN", "DDD", "JPM","GS"]

# Add ticker filter if tickers are provided
if tickers:
    filters.append(('ticker', 'in', tickers))

df = pd.read_parquet(
            f"gs://sovai-partitioned/sovai-accounting/prices.parq",
service_account_info = {}

In [29]:
import io
import json
import pyarrow.dataset as ds
import gcsfs
from google.oauth2 import service_account
from typing import Optional, List, Tuple
import pyarrow as pa
import datetime

def format_filters(ticker_list: List[str], from_date: datetime.datetime, to_date: datetime.datetime) -> List[Tuple]:
    filters = []
    if ticker_list:
        filters.append(('ticker', 'in', ticker_list))
    if from_date and to_date:
        filters.append(('date', '>=', from_date))
        filters.append(('date', '<=', to_date))
    return filters

def common_cloud_storage_stream(uri: str, service_account_info: dict, filters: Optional[List[Tuple]] = None, columns: Optional[List[str]] = None) -> Optional[io.BytesIO]:
    stream = io.BytesIO()

    try:
        scopes = ['https://www.googleapis.com/auth/devstorage.read_write']
        credentials = service_account.Credentials.from_service_account_info(
            service_account_info, scopes=scopes
        )
        gcs_filesystem = gcsfs.GCSFileSystem(token=credentials)
        partitioning = ds.HivePartitioning.discover()
        dataset = ds.dataset(uri, filesystem=gcs_filesystem, partitioning=partitioning)

        filter_conditions = []
        for f in filters:
            if f[1] == 'in':
                filter_conditions.append(ds.field(f[0]).isin(f[2]))
            elif f[1] == '>=':
                filter_conditions.append(ds.field(f[0]) >= f[2])
            elif f[1] == '<=':
                filter_conditions.append(ds.field(f[0]) <= f[2])

        filter_expression = None
        if filter_conditions:
            filter_expression = filter_conditions[0]
            for condition in filter_conditions[1:]:
                filter_expression = filter_expression & condition

        table = dataset.to_table(filter=filter_expression, columns=columns)
        writer = pa.RecordBatchStreamWriter(stream, table.schema)
        writer.write_table(table)
        writer.close()
        stream.seek(0)

        return stream
    except FileNotFoundError:
        return None

# Example usage of the function
uri = 'sovai-partitioned/sovai-accounting/prices.parq'
filters = format_filters(["MSFT", "TSLA", "AMZN", "DDD", "JPM", "GS"], datetime.datetime(2020, 1, 1), datetime.datetime(2020, 12, 31))
stream = common_cloud_storage_stream(uri, service_account_info, filters, None)


In [34]:
import io
import pyarrow.ipc as ipc
import pandas as pd

def read_arrow_stream_to_dataframe(stream: io.BytesIO) -> pd.DataFrame:
    """
    Read a ByteIO stream containing a serialized Arrow table and convert it to a Pandas DataFrame.

    :param io.BytesIO stream: The input stream containing serialized Arrow table.
    :return pd.DataFrame: DataFrame representation of the Arrow table.
    """
    stream.seek(0)
    reader = ipc.open_stream(stream)
    table = reader.read_all().to_pandas()
    if 'ticker' in table.columns:
        cols = ['ticker'] + [col for col in table.columns if col != 'ticker']
        table = table[cols]

    return table

# Assuming 'stream' is the BytesIO object returned from common_cloud_storage_stream
df = read_arrow_stream_to_dataframe(stream)


In [35]:
df

,ticker,date,open,high,low,close,volume,closeadj,closeunadj
2303926,AMZN,2020-01-02,93.750000,94.900002,93.208000,94.900002,80718200.0,94.900002,1898.010010
2303927,AMZN,2020-01-03,93.224998,94.309998,93.224998,93.749001,75332080.0,93.749001,1874.969971
2303928,AMZN,2020-01-06,93.000000,95.183998,93.000000,95.143997,79287456.0,95.143997,1902.880005
2303929,AMZN,2020-01-07,95.224998,95.695000,94.601997,95.343002,82680200.0,95.343002,1906.859985
2303930,AMZN,2020-01-08,94.902000,95.550003,94.321999,94.598999,70239320.0,94.598999,1891.969971
...,...,...,...,...,...,...,...,...,...
38999929,TSLA,2020-12-24,214.330002,222.029999,213.667007,220.589996,68561032.0,220.589996,661.770020
38999930,TSLA,2020-12-28,224.837006,227.132996,220.266998,221.229996,96835680.0,221.229996,663.690002
38999931,TSLA,2020-12-29,220.332993,223.300003,218.332993,221.996994,67592192.0,221.996994,665.989990
38999932,TSLA,2020-12-30,224.000000,232.199997,222.787003,231.593002,128538064.0,231.593002,694.780029


In [12]:
import pyarrow.dataset as ds
import gcsfs
from google.oauth2 import service_account
import json

scopes = ['https://www.googleapis.com/auth/devstorage.read_write']
credentials = service_account.Credentials.from_service_account_info(
    service_account_info, scopes=scopes
)

gcs_filesystem = gcsfs.GCSFileSystem(token=credentials)

# The path to your Parquet file in Google Cloud Storage
file_path = 'sovai-partitioned/sovai-accounting/prices.parq'

# Create Hive partitioning object
partitioning = ds.HivePartitioning.discover()

# Open the Parquet file with the GCSFileSystem object
dataset = ds.dataset(file_path, filesystem=gcs_filesystem, partitioning=partitioning)

# Apply ticker filters
tickers = ["MSFT", "TSLA", "AMZN", "DDD", "JPM", "GS"]
filter_expression = ds.field('ticker').isin(tickers)

# Read the Parquet file into a PyArrow Table
table = dataset.to_table(filter=filter_expression)

# If you want to convert it to a Pandas DataFrame
df = table.to_pandas()


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36050 entries, 2298390 to 39000732
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        36050 non-null  datetime64[ns]
 1   open        36050 non-null  float32       
 2   high        36050 non-null  float32       
 3   low         36050 non-null  float32       
 4   close       36050 non-null  float32       
 5   volume      36050 non-null  float32       
 6   closeadj    36050 non-null  float32       
 7   closeunadj  36050 non-null  float32       
 8   ticker      36050 non-null  category      
dtypes: category(1), datetime64[ns](1), float32(7)
memory usage: 2.2 MB


In [6]:
cols_to_fetch = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRXblp6HCEUbCD9JVfx1abpwU41rbvHNbC9f8KWbj2qKiO7M1VKkUvi9HaVF-jGeXo6wO5tnpanZ9HP/pub?gid=416199758&single=true&output=csv")


In [45]:
df_compensation

,ticker,date,recipient_uei,recipient_name,recipient_parent_name,highly_compensated_officer_1_name,highly_compensated_officer_1_amount,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount
6272,ACCA,2018-10-31,K5TBNBLVG1F8,"ID TECHNOLOGIES, LLC",ACACIA INVESTMENT HOLDINGS LLC,CHRISTOPHER OLIVER,4.855181e+05,DYLAN CONNER,8.205962e+05,GAVIN LONG,5.192971e+05,JEFFERY PANEBIANCO,8.186781e+05,THOMAS BRADY,3.650648e+05
6273,ACCA,2018-11-30,K5TBNBLVG1F8,"ID TECHNOLOGIES, LLC",ACACIA INVESTMENT HOLDINGS LLC,CHRISTOPHER OLIVER,4.567869e+05,DYLAN CONNER,9.454536e+05,MATTHEW OBRIEN,5.966085e+05,JEFFERY PANEBIANCO,9.843824e+05,THOMAS BRADY,3.863974e+05
6274,ACCA,2018-12-31,K5TBNBLVG1F8,"ID TECHNOLOGIES, LLC",ACACIA INVESTMENT HOLDINGS LLC,CHRISTOPHER OLIVER,5.112993e+05,DYLAN CONNER,7.239009e+05,TIMOTHY HEALY,4.608598e+05,BRYAN COURTRIGHT,6.863858e+05,MATTHEW OBRIEN,3.448087e+05
6275,ACCA,2019-01-31,K5TBNBLVG1F8,"ID TECHNOLOGIES, LLC",ACACIA INVESTMENT HOLDINGS LLC,CHRISTOPHER OLIVER,5.431024e+05,DYLAN CONNER,7.199514e+05,GAVIN LONG,4.610935e+05,JEFFERY PANEBIANCO,6.719439e+05,THOMAS BRADY,3.176168e+05
6276,ACCA,2019-02-28,K5TBNBLVG1F8,"ID TECHNOLOGIES, LLC",ACACIA INVESTMENT HOLDINGS LLC,CHRISTOPHER OLIVER,5.139776e+05,DYLAN CONNER,7.543250e+05,GAVIN LONG,4.831822e+05,JEFFERY PANEBIANCO,7.170672e+05,THOMAS BRADY,3.400495e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8816,None,2023-12-31,PK35MEQEASD5,"HEXAGON US FEDERAL, INC",HEXAGON AB,TAMMER M OLIBAH,8.838640e+05,CHRISTOPHER BELLIOS,4.824380e+05,REBECCA D HARRIS,3.976370e+05,LEONARD LEASSEAR,3.151950e+05,CHRISTOPHER REICHERT,3.102680e+05
9263,None,2023-12-31,QEAYP7YMCXZ7,"PRECISE SOFTWARE SOLUTIONS, INC.","IDERA, INC.",ZHENSEN HUANG,3.300000e+05,None,NaN,None,NaN,None,NaN,None,NaN
12331,None,2023-12-31,WSUMT2ADVGJ1,EXECUTIVE PROTECTION SYSTEMS LLC,SENECA HOLDINGS LLC,WARREN KMETZ,2.437670e+05,DAVID MORGAN,1.775810e+05,JOHN HOKE,1.732660e+05,LORI FERNANDES,1.730390e+05,GERARD MCGUIRE,1.611770e+05
13969,None,2023-12-31,ZZRPFWN91KD1,"AT&T TECHNICAL SERVICES COMPANY, INC",AT&T INC.,STACY SCHWARTZ,9.765530e+05,JILL SINGER,1.013555e+06,RICHARD DURAND,5.628760e+05,ELLEN FLOYD,6.169490e+05,KNUTE OLSON,7.323540e+05


In [11]:
!pip install tabulate


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
cols_to_fetch[cols_to_fetch["index"].isin(df_contracts.columns)][["index","Definition"]].to_markdown()

'|     | index                                    | Definition                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |\n|----:|:-----------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
df_contracts = sov.data("spending/contracts", start_date="2008-03-30", tickers=["MSFT","TSLA", "AMZN"], purge_cache=True)

/spending/contracts
Requesting URL: https://fastapi-sovai-y7o724zyfq-uc.a.run.app/spending/contracts with params: {'tickers': 'MSFT,TSLA,AMZN', 'from_date': '2008-03-30T00:00:00'}
Response Status: 200
Response Content-Type: application/octet-stream
tickers is: ['MSFT', 'TSLA', 'AMZN']
{'tickers': 'MSFT,TSLA,AMZN', 'from_date': '2008-03-30T00:00:00'}
tickers is: ['MSFT', 'TSLA', 'AMZN']
Streaming Initialized


In [14]:
df_transactions = sov.data("spending/transactions", start_date="2008-03-30", tickers=["MSFT","TSLA"])

/spending/transactions
Requesting URL: https://fastapi-sovai-y7o724zyfq-uc.a.run.app/spending/transactions with params: {'tickers': 'MSFT,TSLA', 'from_date': '2008-03-30T00:00:00'}
Response Status: 200
Response Content-Type: application/json
tickers is: ['MSFT', 'TSLA']
{'tickers': 'MSFT,TSLA', 'from_date': '2008-03-30T00:00:00'}
tickers is: ['MSFT', 'TSLA']
It reached the DF
It passsed the DF


In [15]:
df_transactions.head()

contract_transaction_unique_key  \
ticker date                                               
MSFT   2008-03-31      9700_9700_8Z03_0_DAAB1502D1002_0   
       2008-03-31  9700_-NONE-_M6739908P0043_0_-NONE-_0   
       2008-03-31    6800_-NONE-_EP08H001236_0_-NONE-_0   
       2008-04-03      9700_9700_F782_0_DAAB1502D1002_0   
       2008-04-03  4735_-NONE-_GS09Q08DF0048_0_-NONE-_0   

                   federal_action_obligation  \
ticker date                                    
MSFT   2008-03-31                   248398.0   
       2008-03-31                     1750.0   
       2008-03-31                   215725.0   
       2008-04-03                   129748.0   
       2008-04-03                    96498.0   

                                    transaction_description  \
ticker date                                                   
MSFT   2008-03-31  CLIN 0129AA AST - MESSAGING TECHNOLOGIES   
       2008-03-31    MICROSOFT IT ACADEMY ANNUAL MEMBERSHIP   
       2008-03-31      MS PREMIER SUPPORT SVCS SUBSCRIPTION   
       2008-04-03           AST- MESSAGING TECHNOLOGY HOURS   
       2008-04-03                 MICROSOFT PREMIER SUPPORT   

                                   contract_award_unique_key  \
ticker date                                                    
MSFT   2008-03-31      CONT_AWD_8Z03_9700_DAAB1502D1002_9700   
       2008-03-31  CONT_AWD_M6739908P0043_9700_-NONE-_-NONE-   
       2008-03-31    CONT_AWD_EP08H001236_6800_-NONE-_-NONE-   
       2008-04-03      CONT_AWD_F782_9700_DAAB1502D1002_9700   
       2008-04-03  CONT_AWD_GS09Q08DF0048_4735_-NONE-_-NONE-   

                    last_modified_date  
ticker date                             
MSFT   2008-03-31  2019-10-16T00:00:00  
       2008-03-31  2017-12-14T00:00:00  
       2008-03-31  2010-03-13T00:00:00  
       2008-04-03  2022-04-08T00:00:00  
       2008-04-03  2008-11-26T00:00:00

In [16]:
cols_to_fetch[cols_to_fetch["index"].isin(df_transactions.columns)][["index","Definition"]].to_markdown()

'|     | index                           | Definition                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |\n|----:|:--------------------------------|:---

In [21]:
df_product = sov.data("spending/product", tickers=["A"])


/spending/product
Requesting URL: https://fastapi-sovai-y7o724zyfq-uc.a.run.app/spending/product with params: {'tickers': 'A'}
An error occurred: 404 Client Error: Not Found for url: https://fastapi-sovai-y7o724zyfq-uc.a.run.app/spending/product?tickers=A


InvalidInputData: {'detail': 'Not Found', 'status_code': 404, 'error': '404 Client Error: Not Found for url: https://fastapi-sovai-y7o724zyfq-uc.a.run.app/spending/product?tickers=A'}

In [23]:
df_product = sov.data("spending/product", tickers=["MSFT","TSLA"])


/spending/product
Requesting URL: https://fastapi-sovai-y7o724zyfq-uc.a.run.app/spending/product with params: {'tickers': 'MSFT,TSLA'}
Response Status: 200
Response Content-Type: application/json
tickers is: ['MSFT', 'TSLA']
{'tickers': 'MSFT,TSLA'}
tickers is: ['MSFT', 'TSLA']
It reached the DF
It passsed the DF


In [24]:
df_product.head()

,ticker,contract_award_unique_key,product_or_service_code_description,naics_code,naics_description,country_of_product_or_service_origin,place_of_manufacture,epa_designated_product,recovered_materials_sustainability
0,TSLA,CONT_AWD_SPE7LX20F345T_9700_SPE7LX18D0042_9700,"CONVERTERS, ELECTRICAL, NONROTATING",335999.0,ALL OTHER MISCELLANEOUS ELECTRICAL EQUIPMENT A...,UNITED STATES,MFG IN U.S.,NOT REQUIRED,NO CLAUSES INCLUDED AND NO SUSTAINABILITY INCL...
1,MSFT,CONT_AWD_0018_2800_SS001360014_2800,IT AND TELECOM- OTHER IT AND TELECOMMUNICATIONS,541512.0,COMPUTER SYSTEMS DESIGN SERVICES,None,NOT A MANUFACTURED END PRODUCT,NOT REQUIRED,NO CLAUSES INCLUDED AND NO SUSTAINABILITY INCLU
2,MSFT,CONT_AWD_HC102818F0119_9700_HC102817D0001_9700,SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,541512.0,COMPUTER SYSTEMS DESIGN SERVICES,UNITED STATES,NOT A MANUFACTURED END PRODUCT,NOT REQUIRED,NO CLAUSES INCLUDED AND NO SUSTAINABILITY INCL...
3,MSFT,CONT_AWD_N0016423FS027_9700_N6600119D0019_9700,SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,541511.0,CUSTOM COMPUTER PROGRAMMING SERVICES,UNITED STATES,NOT A MANUFACTURED END PRODUCT,NOT REQUIRED,NO CLAUSES INCLUDED AND NO SUSTAINABILITY INCL...
4,MSFT,CONT_AWD_HC102818F0847_9700_HC102817D0001_9700,SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,541512.0,COMPUTER SYSTEMS DESIGN SERVICES,UNITED STATES,NOT A MANUFACTURED END PRODUCT,NOT REQUIRED,NO CLAUSES INCLUDED AND NO SUSTAINABILITY INCL...


In [25]:
cols_to_fetch[cols_to_fetch["index"].isin(df_product.columns)][["index","Definition"]].to_markdown()

'|     | index                                | Definition                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |\n|----:|:-------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df_entities = sov.data("spending/entities", tickers=["MSFT","TSLA"])


In [26]:
df_entities.tail()

,ticker,recipient_uei,recipient_parent_uei,recipient_name,recipient_parent_name,names,similarity,bloomberg_share_id
0,TSLA,WK5RYJL58YY9,WK5RYJL58YY9,"TESLA INDUSTRIES, INC.","TESLA INDUSTRIES, INC.","TESLA INDUSTRIES, INC.",0.9562,BBG001SQKGD7
1,MSFT,FMVPEWNJGLM1,FMVPEWNJGLM1,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,1.0000,BBG001S5TD05


In [27]:
cols_to_fetch[cols_to_fetch["index"].isin(df_entities.columns)][["index","Definition"]].to_markdown()

'|     | index                 | Definition                                                                                                                                                                                                                    |\n|----:|:----------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|\n|  43 | recipient_name        | The name of the awardee or recipient that relates to the unique identifier. For U.S. based companies, this name is what the business ordinarily files in formation documents with individual states (when required).          |\n|  46 | recipient_parent_name | The name of the ultimate parent of the awardee or recipient.                                                                                                                                        

In [ ]:
df_location = sov.data("spending/location", tickers=["MSFT","TSLA"])

In [28]:
df_location.tail()

,ticker,contract_award_unique_key,recipient_country_name,recipient_address_line_1,recipient_city_name,recipient_county_name,recipient_state_code,recipient_state_name,recipient_zip_4_code,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_city_name,primary_place_of_performance_county_name,primary_place_of_performance_state_code,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,same_country,same_state
9324,TSLA,CONT_AWD_0542_9700_SPM7MX13D0089_9700,UNITED STATES,109 CENTERPOINT BLVD,NEW CASTLE,NEW CASTLE,DE,DELAWARE,197204180,USA,UNITED STATES,NEW CASTLE,NEW CASTLE,DE,DELAWARE,197204180,1,1
9325,TSLA,CONT_AWD_SPE7M121F0272_9700_SPE7MX18D0028_9700,UNITED STATES,101 109 CENTERPOINT BLVD,NEW CASTLE,NEW CASTLE,DE,DELAWARE,19720,USA,UNITED STATES,NEW CASTLE,NEW CASTLE,DE,DELAWARE,197204180,1,1
9326,TSLA,CONT_AWD_1132_9700_SPM7MX13D0089_9700,UNITED STATES,109 CENTERPOINT BLVD,NEW CASTLE,NEW CASTLE,DE,DELAWARE,197204180,USA,UNITED STATES,NEW CASTLE,NEW CASTLE,DE,DELAWARE,197204180,1,1
9327,TSLA,CONT_AWD_SPE7L723P2381_9700_-NONE-_-NONE-,UNITED STATES,101 109 CENTERPOINT BLVD,NEW CASTLE,NEW CASTLE,DE,DELAWARE,19720,USA,UNITED STATES,NEW CASTLE,NEW CASTLE,DE,DELAWARE,197204180,1,1
9328,TSLA,CONT_AWD_SPE7LX20F734X_9700_SPE7LX18D0042_9700,UNITED STATES,101 109 CENTERPOINT BLVD,NEW CASTLE,NEW CASTLE,DE,DELAWARE,19720,USA,UNITED STATES,NEW CASTLE,NEW CASTLE,DE,DELAWARE,197204180,1,1


In [29]:
cols_to_fetch[cols_to_fetch["index"].isin(df_location.columns)][["index","Definition"]].to_markdown()

'|    | index                                     | Definition                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |\n|---:|:------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df_competition = sov.data("spending/competition", tickers=["MSFT","TSLA"])


In [33]:
df_competition.head()

,ticker,contract_award_unique_key,extent_competed,number_of_offers_received,solicitation_procedures
0,TSLA,CONT_AWD_0162_9700_SPM43003D4053_9700,FULL AND OPEN COMPETITION AFTER EXCLUSION OF S...,2.0,ONLY ONE SOURCE
1,MSFT,CONT_AWD_1025_9700_DAAB1502D1002_9700,NOT COMPETED,1.0,ONLY ONE SOURCE
2,MSFT,CONT_AWD_EP07_9700_DAAB1502D1002_9700,NOT COMPETED,1.0,ONLY ONE SOURCE
3,TSLA,CONT_AWD_0163_9700_SPM43003D4053_9700,FULL AND OPEN COMPETITION AFTER EXCLUSION OF S...,2.0,ONLY ONE SOURCE
4,MSFT,CONT_AWD_SJ03_9700_DAAB1502D1002_9700,NOT COMPETED,1.0,ONLY ONE SOURCE


In [34]:
cols_to_fetch[cols_to_fetch["index"].isin(df_competition.columns)][["index","Definition"]].to_markdown()

'|     | index                     | Definition                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |\n|----:|:--------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [38]:
# df_product = sov.data("spending/product", tickers=["MSFT","TSLA"])
df_compensation = sov.data("spending/compensation")

/spending/compensation
Requesting URL: https://fastapi-sovai-y7o724zyfq-uc.a.run.app/spending/compensation with params: None
Response Status: 200
Response Content-Type: application/json
tickers is None
None
tickers is: 
All ticker Initialized
https://s3.wasabisys.com/sovai-government/postprocessed/df_compensation.park?AWSAccessKeyId=KI07G4DK9XP5EC0NH1VU&Signature=VEffHQPBlJ1OPVy0NLxKHt5rZCA%3D&Expires=1707930780  (download link)


In [43]:
counts

ticker
PGNT     196
GIB      196
STN      195
CADMF    195
ADV      194
        ... 
SAABY      3
UMBF       3
TPPTY      1
TSRI       1
ARAT       1
Length: 94, dtype: int64

In [39]:
df_compensation.head()

,ticker,date,recipient_uei,recipient_name,recipient_parent_name,highly_compensated_officer_1_name,highly_compensated_officer_1_amount,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount
6272,ACCA,2018-10-31,K5TBNBLVG1F8,"ID TECHNOLOGIES, LLC",ACACIA INVESTMENT HOLDINGS LLC,CHRISTOPHER OLIVER,485518.12500,DYLAN CONNER,820596.2500,GAVIN LONG,519297.12500,JEFFERY PANEBIANCO,818678.1250,THOMAS BRADY,365064.81250
6273,ACCA,2018-11-30,K5TBNBLVG1F8,"ID TECHNOLOGIES, LLC",ACACIA INVESTMENT HOLDINGS LLC,CHRISTOPHER OLIVER,456786.87500,DYLAN CONNER,945453.5625,MATTHEW OBRIEN,596608.50000,JEFFERY PANEBIANCO,984382.4375,THOMAS BRADY,386397.40625
6274,ACCA,2018-12-31,K5TBNBLVG1F8,"ID TECHNOLOGIES, LLC",ACACIA INVESTMENT HOLDINGS LLC,CHRISTOPHER OLIVER,511299.34375,DYLAN CONNER,723900.9375,TIMOTHY HEALY,460859.84375,BRYAN COURTRIGHT,686385.8125,MATTHEW OBRIEN,344808.71875
6275,ACCA,2019-01-31,K5TBNBLVG1F8,"ID TECHNOLOGIES, LLC",ACACIA INVESTMENT HOLDINGS LLC,CHRISTOPHER OLIVER,543102.37500,DYLAN CONNER,719951.4375,GAVIN LONG,461093.50000,JEFFERY PANEBIANCO,671943.9375,THOMAS BRADY,317616.84375
6276,ACCA,2019-02-28,K5TBNBLVG1F8,"ID TECHNOLOGIES, LLC",ACACIA INVESTMENT HOLDINGS LLC,CHRISTOPHER OLIVER,513977.62500,DYLAN CONNER,754325.0000,GAVIN LONG,483182.21875,JEFFERY PANEBIANCO,717067.1875,THOMAS BRADY,340049.53125


In [40]:
cols_to_fetch[cols_to_fetch["index"].isin(df_compensation.columns)][["index","Definition"]].to_markdown()

'|     | index                               | Definition                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |\n|----:|:------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

def create_interactive_plots(df_filtered):
    # Create a dropdown widget for ticker selection
    ticker_dropdown = widgets.Dropdown(
        options=df_filtered['ticker'].unique(),
        value='TSLA',
        description='Ticker:',
    )

    # Create a dropdown widget for plot selection
    plot_dropdown = widgets.Dropdown(
        options=['Cumulative Potential Total Value', 'Dynamic Rolling Average Contract Size'],
        value='Cumulative Potential Total Value',
        description='Select Plot:',
    )

    # Create an output widget for displaying the plot
    output = widgets.Output()

    # Function to update and display the selected plot
    def update_plot(ticker, plot_type):
        # Filter for the selected ticker
        df_ticker = df_filtered[df_filtered['ticker'] == ticker]

        # Ensure action_date is a datetime
        df_ticker.loc[:, 'action_date'] = pd.to_datetime(df_ticker['action_date'])

        with output:
            clear_output(wait=True)
            if plot_type == 'Cumulative Potential Total Value':
                # Plot Cumulative Potential Total Value
                df_ticker_monthly = df_ticker.set_index('action_date').resample('ME')['potential_total_value_of_award'].sum().cumsum().values
                fig = px.line(df_ticker_monthly,
                              title='Cumulative Potential Total Value of Award',
                              labels={'value': 'Cumulative Potential Total Value', 'action_date': 'Date'},
                              markers=True)
                fig.update_xaxes(title_text='Date')
                fig.update_yaxes(title_text='Cumulative Potential Total Value')
            elif plot_type == 'Dynamic Rolling Average Contract Size':
                # Calculate the dynamic rolling window size as 5% of the available contracts
                rolling_window_size = int(len(df_ticker) * 0.05)

                # Calculate rolling average contract size with the dynamic window size
                df_ticker['rolling_average_contract_size'] = df_ticker['potential_total_value_of_award'].rolling(window=rolling_window_size).mean()
                fig = px.line(df_ticker,
                              x='action_date',
                              y='rolling_average_contract_size',
                              title='Dynamic Rolling Average Contract Size',
                              labels={'rolling_average_contract_size': 'Rolling Average Contract Size'},
                              markers=True)
                fig.update_xaxes(title_text='Date')
                fig.update_yaxes(title_text='Rolling Average Contract Size')

            # Remove the legend (variable key) on the right-hand side
            fig.update_traces(showlegend=False)

            # Apply the dark theme
            fig.update_layout(template='plotly_dark')

            # Adjust the height of the plot
            fig.update_layout(height=400)

            return fig

    # Create an interactive widget for updating and displaying the plot
    widgets.interactive(update_plot, ticker=ticker_dropdown, plot_type=plot_dropdown)

    # Define the widget_eventhandler for the ticker and plot dropdowns
    def widget_eventhandler(change):
        nonlocal ticker_dropdown, plot_dropdown
        with output:
            clear_output(wait=True)
            ticker = ticker_dropdown.value
            plot_type = plot_dropdown.value
            fig = update_plot(ticker, plot_type)
            fig.show()

    # Add observers to the dropdowns
    ticker_dropdown.observe(widget_eventhandler, names='value')
    plot_dropdown.observe(widget_eventhandler, names='value')

    # Create an HBox container for the dropdowns
    widgets_container = widgets.HBox([ticker_dropdown, plot_dropdown])

    # Initialize the display
    display(widgets_container, output)
    widget_eventhandler(None)
    # update_plot(ticker_dropdown.value, plot_dropdown.value)

# Call the function with your df_filtered
create_interactive_plots(df_filtered)
